In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import *

def initiateSession(): 
    connection_parameters = {
            "account": "ls88978.eu-west-1",
            "user": "snowpark_user",
            "password": "UdemySnowflake12!@",
            "role": "Accountadmin", 
            "warehouse": "compute_wh",
            "database":"SNOWPARK_UDEMY",
            "schema":"TEST_SCHEMA"
    }
    session = Session.builder.configs(connection_parameters).create()
    return session

session = initiateSession()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [3]:
query = "create or replace stage snowpark_udf_stage" +\
        " directory = (enable = true)" +\
        " copy_options = (on_error='skip_file')"
session.sql(query).collect()

[Row(status='Stage area SNOWPARK_UDF_STAGE successfully created.')]

In [4]:
def calculate_deductions(total_price,deduction_percentage):
  return ((deduction_percentage)/100)*total_price

In [5]:
session.udf.register(
    func = calculate_deductions,
    return_type = FloatType(),
    input_types = [FloatType(),FloatType()],
    is_permanent = True,
    name = 'calculate_deductions',
    replace = True,
    stage_location = '@snowpark_udf_stage'
)

In [6]:
sql = """select 
            O_TOTALPRICE,
            calculate_deductions(O_TOTALPRICE,30) 
        from 
            SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS limit 10
        """
session.sql(sql).show()

------------------------------------------------------------
|"O_TOTALPRICE"  |"CALCULATE_DEDUCTIONS(O_TOTALPRICE,30)"  |
------------------------------------------------------------
|119544.06       |35863.218                                |
|7886.05         |2365.815                                 |
|211426.16       |63427.848                                |
|131324.05       |39397.215                                |
|97735.74        |29320.722                                |
|83196.72        |24959.016                                |
|58768.90        |17630.67                                 |
|108138.25       |32441.475                                |
|145446.27       |43633.880999999994                       |
|25240.19        |7572.056999999999                        |
------------------------------------------------------------



In [7]:
@udf(session=session,packages=["pandas"],name="calculate_deductions_decorator",is_permanent = True,replace=True,stage_location = '@snowpark_udf_stage')
def calculate_deductions_decorator(total_price:float,deduction_percentage:float)->float:
  return ((deduction_percentage)/100)*total_price


In [8]:
sql = """select 
            O_TOTALPRICE,
            calculate_deductions_decorator(O_TOTALPRICE,30) 
        from 
            SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS limit 10
        """
session.sql(sql).show()

----------------------------------------------------------------------
|"O_TOTALPRICE"  |"CALCULATE_DEDUCTIONS_DECORATOR(O_TOTALPRICE,30)"  |
----------------------------------------------------------------------
|155918.31       |46775.492999999995                                 |
|99586.08        |29875.824                                          |
|237864.42       |71359.326                                          |
|246278.06       |73883.41799999999                                  |
|274488.56       |82346.568                                          |
|277756.90       |83327.07                                           |
|202932.68       |60879.804                                          |
|146252.59       |43875.776999999995                                 |
|139073.10       |41721.93                                           |
|147367.27       |44210.181                                          |
----------------------------------------------------------------------

